In [1]:
import pandas as pd
import matplotlib.pyplot as plt

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
from nltk.stem import WordNetLemmatizer 
import nltk

import json
import csv

import plotly.graph_objects as go

import string
import sys
import random

stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

import numpy as np
import os

import argparse

from torch.utils.data import TensorDataset
from tqdm import tqdm, trange


In [2]:
if os.getcwd().split('/')[-1] != 'winogrande_1.1':
    os.chdir('/home/bssachde/Winogrande/winogrande_1.1')

In [3]:
data = []
with open('train.jsonl') as f:
    for line in f:
        data.append(json.loads(line))

In [4]:
D=pd.DataFrame(data)
S=D

In [5]:
sentences_list = []
label_list = []
for index in range(0,len(S['sentence'])):
    sentences_list.append(S['sentence'][index].replace('_',S['option1'][index]))
    sentences_list.append(S['sentence'][index].replace('_',S['option2'][index]))
    if S['answer'][index]=='2':
        label_list.append(0)
        label_list.append(1)
    else:
        label_list.append(1)
        label_list.append(0)

In [6]:
# ### The y label will be of the form [ 0 1 ] which implies the second option is the answer ####
# y = []
# for answer_index in range(0,len(S['answer'])):
# #     print(S['answer'][answer_index])
#     if S['answer'][answer_index]=='2':
#         y.append([0,1])
# #         y.append(S['option2'][answer_index])
#     else:
#         y.append([1,0])
# #         y.append(S['option1'][answer_index])

In [7]:
# ### Verify the label format ###
# print(len(y))

In [8]:
## Target Dataset size ##
n = 5000

In [9]:
## Load module for test train split
from sklearn.model_selection import train_test_split

## Load set of Linear models
from sklearn.linear_model import LogisticRegression

## Load non-Linear Models
from sklearn import svm

from transformers import RobertaTokenizer, RobertaForMaskedLM, RobertaModel
import torch

# transformer libraries for fine tune embeddings
from pytorch_transformers import BertPreTrainedModel, RobertaConfig, \
    ROBERTA_PRETRAINED_MODEL_ARCHIVE_MAP, RobertaModel
from pytorch_transformers.modeling_roberta import RobertaClassificationHead
from torch.nn import CrossEntropyLoss



In [10]:
## For distributed training: local_rank
local_rank=-1
no_cuda = False
seed=42
fp16=False
fp16_opt_level='01'

In [11]:
# Setup CUDA, GPU & distributed training
if local_rank == -1 or no_cuda:
    device = torch.device("cuda" if torch.cuda.is_available() and not no_cuda else "cpu")
    n_gpu = torch.cuda.device_count()
else:  # Initializes the distributed backend which will take care of sychronizing nodes/GPUs
    torch.cuda.set_device(local_rank)
    device = torch.device("cuda", local_rank)
    torch.distributed.init_process_group(backend='nccl')
    n_gpu = 1

print(device)

cuda


In [12]:
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
model = RobertaModel.from_pretrained('roberta-base')

model.to(device)

"""                                                                             
     Help you save GPU memory and thus allow bigger models                       
     and bigger batches under the same hardware limitation.
     https://medium.com/the-artificial-impostor/use-nvidia-apex-for-easy-mixed-precision-training-in-pytorch-46841c6eed8c
""" 
if fp16:
    try:
        from apex import amp
    except ImportError:
        raise ImportError("Please install apex from https://www.github.com/nvidia/apex to use fp16 training.")
    model, optimizer = amp.initialize(model, optimizer, opt_level=fp16_opt_level)

if local_rank != -1:
    try:
        from apex.parallel import DistributedDataParallel as DDP
    except ImportError:
        raise ImportError(
            "Please install apex from \
            https://www.github.com/nvidia/apex to use\
            distributed and fp16 training.")

    model = DDP(model)
elif n_gpu > 1 and not no_cuda:
    model = torch.nn.DataParallel(model)

model.eval()
    

RobertaModel(
  (embeddings): RobertaEmbeddings(
    (word_embeddings): Embedding(50265, 768, padding_idx=0)
    (position_embeddings): Embedding(514, 768)
    (token_type_embeddings): Embedding(1, 768)
    (LayerNorm): BertLayerNorm()
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): BertLayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
        )
        (intermediate): BertIntermediate(
          (dense

In [13]:
## This is for pretraining
# !git clone https://github.com/allenai/winogrande.git

In [14]:
# # os.chdir("winogrande/")
# !export PYTHONPATH=$PYTHONPATH:$(pwd)
# !python "scripts/run_experiment.py" --model_type roberta_mc --model_name_or_path roberta-large --task_name winogrande --do_eval --do_lower_case --data_dir ./data  --max_seq_length 80  --per_gpu_eval_batch_size 4  --per_gpu_train_batch_size 16  --learning_rate 1e-5  --num_train_epochs 3  --output_dir ./output/models/  --do_train  --logging_steps 4752  --save_steps 4750  --seed 42  --data_cache_dir ./output/cache/  --warmup_pct 0.1  --evaluate_during_training

In [15]:
print(device)
input_ids = torch.tensor(tokenizer.encode("Hello, my dog is cute", add_special_tokens=True)).unsqueeze(0)  # Batch size 1
input_ids = input_ids.to(device)
print(input_ids.device)
outputs = model(input_ids)#, masked_lm_labels=input_ids)
loss, prediction_scores = outputs[:2]

cuda
cuda:0


In [16]:
output_model_file = "models/pytorch_model.bin"
model_state_dict = torch.load(output_model_file) 
model = RobertaModel.from_pretrained('roberta-large',state_dict=model_state_dict)


model.to(device)

"""                                                                             
     Help you save GPU memory and thus allow bigger models                       
     and bigger batches under the same hardware limitation.
     https://medium.com/the-artificial-impostor/use-nvidia-apex-for-easy-mixed-precision-training-in-pytorch-46841c6eed8c
""" 
if fp16:
    try:
        from apex import amp
    except ImportError:
        raise ImportError("Please install apex from https://www.github.com/nvidia/apex to use fp16 training.")
    model, optimizer = amp.initialize(model, optimizer, opt_level=fp16_opt_level)

if local_rank != -1:
    try:
        from apex.parallel import DistributedDataParallel as DDP
    except ImportError:
        raise ImportError(
            "Please install apex from \
            https://www.github.com/nvidia/apex to use\
            distributed and fp16 training.")

    model = DDP(model)
elif n_gpu > 1 and not no_cuda:
    model = torch.nn.DataParallel(model)

model.eval()
    

RobertaModel(
  (embeddings): RobertaEmbeddings(
    (word_embeddings): Embedding(50265, 1024, padding_idx=0)
    (position_embeddings): Embedding(514, 1024)
    (token_type_embeddings): Embedding(1, 1024)
    (LayerNorm): BertLayerNorm()
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=1024, out_features=1024, bias=True)
            (key): Linear(in_features=1024, out_features=1024, bias=True)
            (value): Linear(in_features=1024, out_features=1024, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=1024, out_features=1024, bias=True)
            (LayerNorm): BertLayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
        )
        (intermediate): BertIntermediate(
     

In [17]:
outputs = model(input_ids)#, masked_lm_labels=input_ids)
print(os.getcwd())
listOfOutputs = [outputs[1][0],outputs[1][0]]
# print(listOfOutputs)

# with open("exp.data","wb") as f:
#     torch.save(outputs[1][0],f)
## clean the file
open("exp.data", 'w').close()
for element in listOfOutputs:
    with open("exp.data","ab") as f:
        torch.save(element,f)
    

/home/bssachde/Winogrande/winogrande_1.1


In [18]:
with open("exp.data","rb") as f:
    while 1:
        try:
            print(torch.load(f))
        except EOFError as error:
            # Output expected EOFErrors.
            break
        except Exception as exception:
            # Output unexpected Exceptions.
            print("Unexpected error:", sys.exc_info())
        

tensor([ 0.1466,  0.7200,  0.4421,  ..., -0.0582,  0.4900, -0.4155],
       device='cuda:0', requires_grad=True)
tensor([ 0.1466,  0.7200,  0.4421,  ..., -0.0582,  0.4900, -0.4155],
       device='cuda:0', requires_grad=True)


In [19]:
# ### The model basically returns the top layer embeddings and the pooled layer ###
# for element in X:
#     print(element[1])

In [20]:
# # ### We are using pickle to save the progress in a file ###
# import pickle
# from torch.utils.data import TensorDataset

# file_name = 'listfile_pooled_9.data'
# X_prime = []
# ## Check if there was already some progress then continue from there otherwise just start from begining
# if not os.path.isfile("start_pooled_9.data"):
#     start = 0
# else:
#     with open("start_pooled_9.data","r") as fileHandle:
#         start = fileHandle.readline()
#         # Scenario if file is there but nothing is in the file
#         if start == '':
#             start = 0
            
# # for index in range(int(start),len(S['sentence'])):
# for index in range(int(start),len(sentences_list)):
#     if index%50==0:
#         print("Saving the progress for embeddings")
#         with open(file_name, 'a+b') as filehandle:
#             pickle.dump(X_prime, filehandle, pickle.HIGHEST_PROTOCOL)
#         ## Reset the embeddings list X_prime and just keep appending the data in the file
#         X_prime = []
#         ## overwrite the index so far in seprate file
#         with open("start_pooled_9.data","w") as fileHandle:
#             fileHandle.write(str(index))
#     print(index)
# #     input_ids = torch.tensor(tokenizer.encode(S['sentence'][index], add_special_tokens=True)).unsqueeze(0)  # Batch size 1
#     input_ids = torch.tensor(tokenizer.encode(sentences_list[index], add_special_tokens=True)).unsqueeze(0)
#     input_ids = input_ids.to(device)
# #     X_prime.append(TensorDataset(model(input_ids)[1][0]))#, masked_lm_labels=input_ids)
#     X_prime.append(model(input_ids)[1][0])#, masked_lm_labels=input_ids)
    
# ### We have to save one more time after the loop is over otherwise the last 50 value won't be saved just calculated ###    
# print("Saving the progress for embeddings")
# with open(file_name, 'a+b') as filehandle:
#     pickle.dump(X_prime, filehandle, pickle.HIGHEST_PROTOCOL)


In [21]:
# # # ### We are using pickle to save the progress in a file ###
# # import pickle

# X_prime = []
# batch_size = 1

# def batch(iterable, n=1):
#     l = len(iterable)
#     for ndx in range(0, l, n):
#         yield iterable[ndx:min(ndx + n, l)]
        
# # for index in range(int(start),len(S['sentence'])):
# for batch_of_sentences in batch(sentences_list, batch_size):
# #     print(batch_of_sentences)
#     input_ids = torch.tensor(tokenizer.encode(batch_of_sentences, add_special_tokens=True)).unsqueeze(0)
#     input_ids = input_ids.to(device)
    
#     TensorDataset(input_ids)
    
#     X_prime.append(model(input_ids)[1][0])#, masked_lm_labels=input_ids)
    

In [22]:
# ### Now we have to get the pooled embeddings for both the options ### 

# # y_emb_option_1 = []
# y_emb_option_2 = []


# ## region ## This is for mainting the position where the code was inturrupted due to any reason what so ever ##
# if not os.path.isfile("start_y_pooled.data"):
#     start = 0
# else:
#     with open("start_y_pooled.data","r") as fileHandle:
#         start = fileHandle.readline()
#         # Scenario if file is there but nothing is in the file
#         if start == '':
#             start = 0
# ## end region 

# # ## getting embedding for option 1
# # for index in range(0,len(S['option1'])):
# #     if index%50==0 and index!=0:
# #         print("Saving the progress for embeddings")
# #         with open('y_label_pooled_option1.data', 'a+b') as filehandle:
# #             pickle.dump(y_emb_option_1, filehandle, pickle.HIGHEST_PROTOCOL)
# #         ## Reset the embeddings list X_prime and just keep appending the data in the file
# #         y_emb_option_1 = []
# #         ## overwrite the index so far in seprate file
# #         with open("start_y_pooled.data","w") as fileHandle:
# #             fileHandle.write(str(index))
# #     print(index)
# #     input_ids = torch.tensor(tokenizer.encode(S['option1'][index], add_special_tokens=True)).unsqueeze(0)  # Batch size 1
# # #     print(input_ids)
# #     output_ids = model(input_ids)[1][0]
# #     y_emb_option_1.append(output_ids)#, masked_lm_labels=input_ids)
    
# # print("Saving the progress for embeddings")
# # with open('y_label_pooled_option1.data', 'a+b') as filehandle:
# #     pickle.dump(y_emb_option_1, filehandle, pickle.HIGHEST_PROTOCOL)
# # ## overwrite the index so far in seprate file
# # with open("start_y_pooled.data","w") as fileHandle:
# #     fileHandle.write(str(index))


# ## getting embedding for option 2
# for index in range(int(start),len(S['option2'])):
#     if index%50==0 and index!=0:
#         print("Saving the progress for embeddings")
#         with open('y_label_pooled_option2.data', 'a+b') as filehandle:
#             pickle.dump(y_emb_option_2, filehandle, pickle.HIGHEST_PROTOCOL)
#         ## Reset the embeddings list X_prime and just keep appending the data in the file
#         y_emb_option_1 = []
#         ## overwrite the index so far in seprate file
#         with open("start_y_pooled.data","w") as fileHandle:
#             fileHandle.write(str(index))
#     print(index)
#     input_ids = torch.tensor(tokenizer.encode(S['option2'][index], add_special_tokens=True)).unsqueeze(0)  # Batch size 1
# #     print(input_ids)
#     output_ids = model(input_ids)[1][0]
#     y_emb_option_2.append(output_ids)#, masked_lm_labels=input_ids)
    
# print("Saving the progress for embeddings")
# with open('y_label_pooled_option2.data', 'a+b') as filehandle:
#     pickle.dump(y_emb_option_2, filehandle, pickle.HIGHEST_PROTOCOL)
# ## overwrite the index so far in seprate file
# with open("start_y_pooled.data","w") as fileHandle:
#     fileHandle.write(str(index))
        

In [23]:
### verify the Embeddings for options ###
# print(y_emb_option_2)

In [24]:
print(os.getcwd())
os.path.getsize('listfile_pooled_9.data')

/home/bssachde/Winogrande/winogrande_1.1


87796814

In [25]:
X = []
# y_option1 = []
# y_option2 = []

In [26]:
import sys
import pickle

with open('listfile_pooled_9.data', mode='b+r') as f:
    while 1:
        try:
            X.append(pickle.load(f))
        except EOFError as error:
            # Output expected EOFErrors.
            break
        except Exception as exception:
            # Output unexpected Exceptions.
            print("Unexpected error:", sys.exc_info())
            continue

# with open('y_label_pooled_option1.data', mode='br') as f:
#     while 1:
#         try:
#             y_option1.append(pickle.load(f))
#         except:
#             break
# with open('y_label_pooled_option2.data', mode='br') as f:
#     while 1:
#         try:
#             y_option2.append(pickle.load(f))
#         except:
#             break
            
### The final features look something like follows:
### <Sentence embeddings> <Option 1> <Sentence embeddings> <Option 2>

In [27]:
# print(X)
X_prime = []
flat_list = []
for sublist in X:
    for item in sublist:
        flat_list.append(item.detach().cpu().numpy())

X_prime = flat_list[::]
# ## make a copy for the Sentence embeddings 
# X_copy = flat_list[::]            

# flat_list = []
# for sublist in y_option1:
#     for item in sublist:
#         flat_list.append(item.detach().numpy())

# y_option1 = flat_list[::]

# flat_list = []
# for sublist in y_option2:
#     for item in sublist:
#         flat_list.append(item.detach().numpy())

# y_option2 = flat_list[:10000]

In [28]:
print(len(X_prime))
print(len(label_list))
print(X_prime)
# print(S['sentence'][0])
# X_prime = []
# for element in X:
#     X_prime.append(np.asarray(element[0][0]))
# X_prime = []
# for x1,y1,x2,y2 in zip(X,y_option1,X_copy,y_option2):
# #     print(len(x1))
# #     print(len(y1))
# #     print(len(x2))
# #     print(len(y2))
#     temp = [*x1,*y1,*x2,*y2]
# #     print(len(temp))
#     X_prime.append(temp)
    

20000
20000
[array([ 0.24725541,  0.70067024,  0.3812275 , ..., -0.15011704,
        0.5687383 , -0.50533247], dtype=float32), array([ 0.23328483,  0.70485747,  0.3925203 , ..., -0.14027862,
        0.56054753, -0.4946096 ], dtype=float32), array([ 0.24531335,  0.6978612 ,  0.39882913, ..., -0.14773904,
        0.5701736 , -0.48274782], dtype=float32), array([ 0.24876365,  0.70064414,  0.39933097, ..., -0.13861734,
        0.568116  , -0.48823068], dtype=float32), array([ 0.2367833 ,  0.67119384,  0.436176  , ..., -0.13477737,
        0.5652629 , -0.4906109 ], dtype=float32), array([ 0.2315505 ,  0.6736901 ,  0.43745726, ..., -0.13496311,
        0.5658487 , -0.4892431 ], dtype=float32), array([ 0.23098709,  0.6766957 ,  0.4424159 , ..., -0.12815228,
        0.57488465, -0.4923432 ], dtype=float32), array([ 0.23142816,  0.67792094,  0.44273457, ..., -0.12849471,
        0.5745969 , -0.491496  ], dtype=float32), array([ 0.24400257,  0.6704524 ,  0.46257922, ..., -0.08232059,
        0.5

In [29]:
# # for element in X_prime:
# #     print(len(element))
# # print(y.shape)
# X_temp = np.asarray(X_prime)

In [30]:
X_temp = np.asarray(X_prime)
y_temp = np.asarray(label_list)

In [31]:
# y_prime = []
# for element in y:
#     y_prime.append(np.asarray(element[0][0]))

In [32]:
# for element in y_prime:
#     print(element.shape)
    
# y_double_prime = S['answer'][:-50]

In [33]:
# y_double_prime

In [ ]:
from skmultilearn.problem_transform import LabelPowerset

def find_index_of_array(list, array):
    for i in range(len(list)):
        if np.all(list[i]==array):
            return i
# while len(S) > n:
#Dictionary to save the mdoel predictions for every individual elements.
E = dict.fromkeys(S['qID'])
EPrime = dict.fromkeys(S['sentence'])

D = {}
results = {}
# print(D)
m=4
for i in range(0,m):
    print("Iteration : "+str(i))
    ## 90 percent of the data is in the test set
    
    print("Training Samples: ")
#     print(X_temp[:10])
#     print(y_temp[:10])
    
    print("Test train Split")
    X_train, X_test, y_train, y_test = train_test_split(X_temp,y_temp, test_size=0.10, random_state=42)
    
    print("Training Set Samples: ")
    print(X_train[:10])
    print(len(X_train))
    print(len(X_test))
    print(len(y_train))
    print(len(y_test))
    
    ## Train models
    ### Logistic Regression
    lm = LabelPowerset(LogisticRegression(random_state=0,C=0.01,penalty = 'l2',max_iter=400))
    lm.fit(X_train, y_train)
    print("Logistic regression training complete")
    
    
    ### SVM
    rbf_svc = LabelPowerset(svm.SVC(kernel='rbf'))
    rbf_svc.fit(X_train,y_train)
    print("SVM training complete")

    # Predictions
    
#     print(X_test)
    predictions = lm.predict(X_test)
    print("Done with the predictions for LR")
    predictions_svm = rbf_svc.predict(X_test)
    print("Done with the predictions for SVM")
#     print(predictions_svm)
    
    print(predictions[:10])
    
# #     X_test_dict = {}
#     for i in range(0,len(X_test)):
# #         print(X_test[i])
#         index = find_index_of_array(X,X_test[i])
# #         print(index)
#         if index not in D.keys():
#             D[index] = [predictions[i],predictions_svm[i]]
#         else:
#             D[index].append(predictions[i])
#             D[index].append(predictions_svm[i])
#         results[index] = y_test[i]
        

Iteration : 0
Training Samples: 
Test train Split
Training Set Samples: 
[[ 0.22444904  0.6791436   0.4122615  ... -0.14595328  0.5557538
  -0.48516724]
 [ 0.25523317  0.68864894  0.3856248  ... -0.14182077  0.55401134
  -0.50509644]
 [ 0.22864811  0.6964315   0.42389566 ... -0.13917701  0.55249524
  -0.50798166]
 ...
 [ 0.24098766  0.70607316  0.41566804 ... -0.12833431  0.5527788
  -0.500468  ]
 [ 0.21279745  0.69077975  0.41537738 ... -0.1439547   0.5590444
  -0.5066494 ]
 [ 0.21178453  0.68279743  0.37426227 ... -0.13694127  0.51813745
  -0.5069491 ]]
18000
2000
18000
2000
Logistic regression training complete
SVM training complete
Done with the predictions for LR
Done with the predictions for SVM
  (1, 0)	1
  (2, 0)	1
  (5, 0)	1
  (8, 0)	1
  (9, 0)	1
Iteration : 1
Training Samples: 
Test train Split
Training Set Samples: 
[[ 0.22444904  0.6791436   0.4122615  ... -0.14595328  0.5557538
  -0.48516724]
 [ 0.25523317  0.68864894  0.3856248  ... -0.14182077  0.55401134
  -0.50509644]


In [35]:
# #predictibility score
# P = {}
# for key,value in D.items():
#     print(key)
# #     print(results[key])
# #     print(value.count(results[key])/len(value))
# #     P[key] = value.count(results[key])/len(value)
# # print(P)
# #     print(predictions)   

In [36]:
D

{}